In [ ]:
pip install pdfminer.six


In [ ]:
pip install PyPDF2


In [15]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)
        for page_num in range(num_pages):
            page_text = pdf_reader.pages[page_num].extract_text()
            if page_text:
                text += page_text
    return text


pdf_path = 'kosdaq.pdf'


extracted_text = extract_text_from_pdf(pdf_path)


In [ ]:
print(extracted_text)

In [39]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    try:
        # PDF 파일 열기
        pdf_document = fitz.open(pdf_path)
        
        # 텍스트 추출을 위한 빈 문자열
        text = ""
        
        # 각 페이지에 대해 텍스트 추출
        for page_num in range(len(pdf_document)):
            page_text = pdf_document[page_num].get_text()
            text += page_text + "\n"  # 각 페이지 텍스트를 한 줄로 합침
        
        return text
        
    except Exception as e:
        print(f"PDF 파일을 읽는 중 오류가 발생했습니다: {e}")
        return None

def extract_company_names(text):
    lines = text.split('\n')  # 텍스트를 라인으로 분할
    
    company_names = []
    
    # 각 라인에서 회사명 추출
    for line in lines:
        parts = line.split()
        if len(parts) > 0:
            company_name = parts[0]
            company_names.append(company_name)
    
    return company_names

# PDF 파일 경로
pdf_path = "kosdaq.pdf"

# PDF 파일에서 텍스트 추출
extracted_text = extract_text_from_pdf(pdf_path)

if extracted_text:
    # 추출된 텍스트에서 회사명 추출
    company_names = extract_company_names(extracted_text)
    
    # 결과 출력
    print("추출된 회사명:")
    for name in company_names:
        print(name)
else:
    print("PDF 파일을 읽을 수 없습니다.")


추출된 회사명:
회사명
종목코드
업종
아이엠비디엑스
461030
자연과학
하나32호스팩
475240
금융
엔젤로보틱스
455900
특수
제이투케이바이오
420570
기타
삼현
437730
자동차
오상헬스케어
036220
의료용품
케이엔알시스템
199430
측정,
제조업;
비엔케이제2호스팩
473370
금융
하나31호스팩
469900
금융
SK증권제11호스팩
472230
금융
유안타제15호스팩
473050
금융
유진스팩10호
468760
금융
이에이트
418620
소프트웨어
코셈
360350
측정,
제조업;
케이웨더
068100
일반
사피엔반도체
452430
반도체
에스피소프트
443670
컴퓨터
관리업
스튜디오삼익
415380
기타
신영스팩10호
472220
금융
폰드그룹
472850
봉제의복
IBKS제24호스팩
469480
금융
레이저옵텍
199550
의료용
이닉스
452400
자동차
포스뱅크
105760
일반
현대힘스
460930
선박
HB인베스트먼트
440290
기타
드림인사이트
362990
광고업
대신밸런스제17호스팩
471050
금융
우진엔텍
457550
그외
한빛레이저
452190
특수
IBKS제23호스팩
467930
금융
하나30호스팩
469880
금융
씨싸이트
109670
봉제의복
블루엠텍
439580
기타
LS머트리얼즈
417200
전자부품
케이엔에스
432470
특수
교보15호스팩
465320
금융
와이바이오로직스
338840
자연과학
삼성스팩9호
468510
금융
에이텀
355690
전자부품
엔에이치스팩30호
466910
금융
에이에스텍
453860
기타
그린리소스
402490
반도체
제이엔비
452160
특수
한선엔지니어링
452280
기타
에코아이
448280
기타
스톰테크
352090
플라스틱제품
캡스톤파트너스
452300
기타
에스와이스틸텍
365330
구조용
제조업
에이직랜드
445090
반도체
한국제13호스팩
464440
금융
메가터치
446540
일차전지
비아이매트릭스
413640
소프트웨어
컨텍
451760
컴퓨터
관리업
큐로

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import csv
import time

# 웹 드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 웹 페이지 로드
url = "http://kpat.kipris.or.kr/kpat/searchLogina.do?next=MainSearch#page1"
driver.get(url)


wait = WebDriverWait(driver, 3)  

# 빈 리스트 생성
company_list = []
search_result_list = []

# 이전에 검색된 결과를 저장하기 위한 세트
previous_results = set()


with open('./kosdaq기업목록.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        company_name = row[0]

        try:
            
            search_input = wait.until(EC.presence_of_element_located((By.ID, 'queryText')))
            search_input.clear()
            search_input.send_keys("*AP=[" + company_name + "]")
            search_input.send_keys(Keys.ENTER)
            print("회사명:", company_name)

           
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.search_txt')))

           
            search_results = [element.text for element in driver.find_elements(By.CSS_SELECTOR, '.search_txt')]
            if not search_results:
                search_results = ["검색 결과가 없습니다."]

            
            unique_results = list(set(search_results) - previous_results)
            company_list.append(company_name)
            search_result_list.append(unique_results)
            previous_results.update(unique_results)  

           
            time.sleep(3)  # 3초 대기

        except Exception as e:
            print(f"검색 결과를 가져오는 동안 예외 발생 ({company_name}):", e)
            company_list.append(company_name)
            search_result_list.append(["검색 오류"])


df = pd.DataFrame({'회사명': company_list, '검색 결과': search_result_list})


df.to_csv('검색결과12.csv', index=False)


driver.quit()


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import csv
import time


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


url = "http://kpat.kipris.or.kr/kpat/searchLogina.do?next=MainSearch#page1"
driver.get(url)

wait = WebDriverWait(driver, 3)


company_list = []
formatted_search_results_list = []  

with open('./kosdaq기업목록.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        company_name = row[0]
    
        time.sleep(2)

        try:
            # 검색 입력란을 찾고, 회사명을 입력한 뒤 검색합니다.
            search_input = wait.until(EC.presence_of_element_located((By.ID, 'queryText')))
            search_input.clear()
            search_input.send_keys("*AP=[" + company_name + "]")
            search_input.send_keys(Keys.ENTER)
            print("회사명:", company_name)

            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.search_txt')))

            
            search_results = driver.find_elements(By.CSS_SELECTOR, '.search_txt')
            filing_dates = driver.find_elements(By.CSS_SELECTOR, '#mainsearch_info_list > div.mainlist_topinfo > li:nth-child(3) > span.point01 > a')

            formatted_search_results = []
            for result, date in zip(search_results, filing_dates):
                formatted_result = f"{date.text}: {result.text}"
                formatted_search_results.append(formatted_result)

            if not formatted_search_results:
                formatted_search_results = ["검색 결과가 없습니다."]

            company_list.append(company_name)
            formatted_search_results_list.append(formatted_search_results)

           
            time.sleep(2)

        except Exception as e:
            print(f"검색 결과를 가져오는 동안 예외 발생 ({company_name}):", e)
            company_list.append(company_name)
            formatted_search_results_list.append(["검색 오류"])

       
        search_input.clear()

# 데이터프레임 생성
df = pd.DataFrame({'회사명': company_list, '검색 결과 (출원일자 포함)': formatted_search_results_list})


df.to_csv('검색결과_출원일자포함3.csv', index=False)


driver.quit()


회사명: 아이엠비디엑스
회사명: 하나32호스팩
회사명: 엔젤로보틱스
회사명: 제이투케이바이오
회사명: 삼현
회사명: 오상헬스케어
회사명: 케이엔알시스템
회사명: 비엔케이제2호스팩
회사명: 하나31호스팩
검색 결과를 가져오는 동안 예외 발생 (하나31호스팩): Message: 
Stacktrace:
	GetHandleVerifier [0x00E34CA3+225091]
	(No symbol) [0x00D64DF1]
	(No symbol) [0x00C09A7A]
	(No symbol) [0x00C4175B]
	(No symbol) [0x00C4188B]
	(No symbol) [0x00C77882]
	(No symbol) [0x00C5F5A4]
	(No symbol) [0x00C75CB0]
	(No symbol) [0x00C5F2F6]
	(No symbol) [0x00C379B9]
	(No symbol) [0x00C3879D]
	sqlite3_dbdata_init [0x012A9A43+4064547]
	sqlite3_dbdata_init [0x012B104A+4094762]
	sqlite3_dbdata_init [0x012AB948+4072488]
	sqlite3_dbdata_init [0x00FAC9A9+930953]
	(No symbol) [0x00D707C4]
	(No symbol) [0x00D6ACE8]
	(No symbol) [0x00D6AE11]
	(No symbol) [0x00D5CA80]
	BaseThreadInitThunk [0x778FFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77A97C5E+286]
	RtlGetAppContainerNamedObjectPath [0x77A97C2E+238]

회사명: SK증권제11호스팩
검색 결과를 가져오는 동안 예외 발생 (SK증권제11호스팩): Message: 
Stacktrace:
	GetHandleVerifier [0x00E34CA3+225091]
	(No symb